In [ ]:
number_of_steps = 1000
cutoff_step_limit = 1e-4
cutoff_steps    = 20

In [ ]:
import random 
import os
import numpy as np

In [ ]:
from Digits_StructureText_writer import StructureTextWriter
from Digits_freq_writer import Frequncies_writer
from Digits_MumaxFile_Writer import MumaxFileWriter
from Digit_Intensity_Calculator import IntensityCalculation #set path here
from Digit_Intensity_Calculator import calculate_intensities_all_arms

In [ ]:
def intensity_calculation(struct, step_name):
    #Writing structure file
    struct_text = StructureTextWriter(struture_array=np.array(struct))
    struct_text.square_removel()
    struct_text.struture_text
    
    f = open('overview_S{}.txt'.format(step_name),'w')
    position_of_max = []
    
    ftt_func = []
    for digits in range(3):
        freq_writer = Frequncies_writer(digit=digits)
        freq_writer.write_frequncies()
        freq_text = freq_writer.frequncy_string + freq_writer.mask_string

        inputfile = 's{}f{}'.format(step_name, digits)
        mumaxfile = MumaxFileWriter(
            filename='Digit_NN_allsides_512x128_OOP_0mT.mx3',
            structre_text= struct_text.struture_text,
            frequncy_text= freq_text,
            writefilename= inputfile+'.mx3'
                    )
        mumaxfile.file_writing()
        run_command = 'mumax3 {}'.format(inputfile+'.mx3')
        os.system(run_command)

        #calculating the intensities at all arms
        all_intensities = calculate_intensities_all_arms(inputfile+'.out/m000002.ovf')
        ftt_func.append(all_intensities[digits])
        position_of_max.append(all_intensities.index(max(all_intensities)))
        f.write('digit{} '.format(digits)+ str(all_intensities) + '\n')
        ftt_func.append(random.random())
    f.close()
    return sum(ftt_func)

In [ ]:
def generate_first_structure(length: int):
    return random.choices([0, 1], weights=[85,15], k=length)

In [ ]:
def performing_change(position1Darray):
    
    position1Darray = np.array(position1Darray)
    random1 = random.choice(np.where(position1Darray == 1)[0])
    random0 = random.choice(np.where(position1Darray == 0)[0])
    
    position1Darray[random1] = 0
    position1Darray[random0] = 1
        
    return position1Darray        

In [ ]:
all_struct         = []
accept_struct      = []
all_intensities    = []
accept_intensities = []
changes_made       = 0

for ii in range(number_of_steps):
    first_struct       = generate_first_structure(length=100)
    
    if ii==0:
        intensity = intensity_calculation(first_struct, ii)
        accept_intensities.append(intensity)
        accept_struct.append(first_struct)
        all_intensities.append(intensity)
        all_struct.append(first_struct)
        
    else:
        new_struct = performing_change(accept_struct[-1])
        intensity = intensity_calculation(new_struct, ii)
        
        if intensity>accept_intensities[-1]:
            accept_intensities.append(intensity)
            accept_struct.append(new_struct)
            
            all_intensities.append(intensity)
            all_struct.append(new_struct)
            changes_made = 0
            
        else:
            all_intensities.append(intensity)
            all_struct.append(new_struct)
            changes_made += 1
            
    if changes_made>cutoff_steps:
        break